In [1]:
print("Hello world")

Hello world


In [1]:
import tweepy

token = "" #bearer token, toegang als developer
client = tweepy.Client(bearer_token=token)

userid = "7174972" #twitter id, converter
paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=8) #https://docs.tweepy.org/en/stable/client.html

for tweet in paginator.flatten(limit=8): #Acht resultaten
    print(tweet)


Met een slag op de gong start de CHE vandaag het collegejaar 2022/2023! Aangevuurd door de keynote van oud-minister @arieslob, die de CHE en haar partners aanvuurde de aanstormende #GeneratieZ te omarmen én te gidsen bij hun start op de arbeidsmarkt: Happy new year! https://t.co/8GtSTVpozn
Vandaag congres 'Hart voor de zaak - geloven in je werk', resultaat van een vierjarig gezamenlijk onderzoek door de Christelijke Hogeschool Ede, Lelie Zorggroep en Woord en Daad. 

Resultaten zijn samengevat in een prachtig publieksmagazine: https://t.co/HREQicb2D3 https://t.co/g9TTOTkh9Z
Ben je een ervaren adviseur online communicatie &amp; marketing? Dan heeft de CHE een leuke uitdagende baan voor je! https://t.co/OXtQcir7pa
Aankomend weekend vind je de CHE op Pinksterconferentie Opwekking. Onze studenten en docenten staan met een leuke stand op de Mission Plaza. Kom je ook langs?
 
Op de foto? 📸
In onze stand staan we met een leuke photobooth. Mis ons niet! En kom langs voor een leuke herinnering 